Cryptocurrency trading bot.

In [1]:
##%pip install python-binance==0.7.5
#%pip install --upgrade python-binance
#%pip install paramiko

## API key management.

In [2]:
from cryptocurrency.authentication import Cryptocurrency_authenticator
authenticator = Cryptocurrency_authenticator(use_keys=False, testnet=False)
client = authenticator.spot_client
client

## Get all available pairs for information needed to trade.

In [3]:
from cryptocurrency.exchange import Cryptocurrency_exchange
exchange = Cryptocurrency_exchange(client=client, directory='crypto_logs')
exchange_info = exchange.info
exchange_info

,symbol,base_asset,base_asset_precision,quote_asset,quote_precision,quote_asset_precision,base_asset_commission,quote_commission_precision,allow_trailing_stop,cancel_replace_allowed,min_price,max_price,tick_size,step_size,multiplier_up,multiplier_down
0,ETHBTC,ETH,8,BTC,8,8,8,8,True,True,1.000000e-06,922327.0,1.000000e-06,0.0001,5.0,0.2
1,LTCBTC,LTC,8,BTC,8,8,8,8,True,True,1.000000e-06,100000.0,1.000000e-06,0.0010,5.0,0.2
2,BNBBTC,BNB,8,BTC,8,8,8,8,True,True,1.000000e-06,100000.0,1.000000e-06,0.0010,5.0,0.2
3,NEOBTC,NEO,8,BTC,8,8,8,8,True,True,1.000000e-06,100000.0,1.000000e-06,0.0100,5.0,0.2
4,QTUMETH,QTUM,8,ETH,8,8,8,8,True,True,1.000000e-06,1000.0,1.000000e-06,0.1000,5.0,0.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1429,OSMOUSDT,OSMO,8,USDT,8,8,8,8,True,True,1.000000e-03,1000.0,1.000000e-03,0.0100,5.0,0.2
1430,OSMOBUSD,OSMO,8,BUSD,8,8,8,8,True,True,1.000000e-03,1000.0,1.000000e-03,0.0100,5.0,0.2
1431,HFTBTC,HFT,8,BTC,8,8,8,8,True,True,1.000000e-08,1.0,1.000000e-08,0.1000,5.0,0.2
1432,HFTBUSD,HFT,8,BUSD,8,8,8,8,True,True,1.000000e-04,1000.0,1.000000e-04,0.1000,5.0,0.2


## Get conversion table for price conversions.

In [4]:
from_asset = 'BDOT'
to_asset = 'BETH'

from cryptocurrency.conversion import get_shortest_pair_path_between_assets

shortest_pair_path_between_assets = \
    get_shortest_pair_path_between_assets(from_asset, to_asset, 
                                          exchange_info, 
                                          priority='fees')

shortest_pair_path_between_assets

[('BDOT', 'DOT'), ('DOT', 'BUSD'), ('BETH', 'BUSD')]

In [5]:
from cryptocurrency.conversion_table import get_conversion_table
conversion_table = get_conversion_table(client=client, exchange_info=exchange_info, as_pair=True)
conversion_table

,symbol,base_asset,quote_asset,is_shorted,price_change_percent,weighted_average_price,open,high,low,close,...,rolling_traded_volume,traded_bid_volume,traded_ask_volume,traded_price,traded_bid_price,traded_ask_price,bid_ask_percent_change,bid_ask_volume_percent_change,traded_bid_ask_percent_change,traded_bid_ask_volume_percent_change
close_time,,,,,,,,,,,,,,,,,,,,,
2022-11-13 14:55:27.446,BUSDVAI,BUSD,VAI,False,0.288,1.04080448,1.040000e+00,1.046000e+00,1.040000e+00,1.043000e+00,...,4.187645e+09,3.475899e+07,6.642496e+07,649.144002,NaN,NaN,0.383142,47.373961,NaN,34.352280
2022-11-13 16:05:18.499,BUSDZAR,BUSD,ZAR,False,0.000,18.13402817,1.813000e+01,1.822000e+01,1.807000e+01,1.813000e+01,...,4.187645e+09,3.475899e+07,6.642496e+07,649.144002,NaN,NaN,0.110314,99.942726,NaN,34.352280
2022-11-13 16:05:19.859,TKOBIDR,TKO,BIDR,False,0.953,3220.50,3.019290e+03,3.500000e+03,2.981000e+03,3.048070e+03,...,2.207907e+06,1.303381e+05,9.695476e+04,0.195000,46.030381,46.258931,0.493603,57.230360,0.494068,57.343679
2022-11-13 16:05:33.482,QKCBTC,QKC,BTC,False,0.000,0.00000045,4.500000e-07,4.700000e-07,4.300000e-07,4.500000e-07,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,2.222222,54.461160,NaN,NaN
2022-11-13 16:06:31.154,BNTBUSD,BNT,BUSD,False,-3.261,0.36362816,3.680000e-01,3.750000e-01,3.530000e-01,3.560000e-01,...,4.085036e+05,1.051709e+03,4.580203e+02,0.356000,0.105954,0.106040,0.840336,43.476742,0.080563,69.662099
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-11-13 16:10:41.174,MATICUSDT,MATIC,USDT,False,-8.605,0.91570170,9.564000e-01,9.686000e-01,8.648000e-01,8.741000e-01,...,1.844472e+08,2.076241e+05,1.695970e+05,0.874100,50.690071,50.796747,0.011435,18.173777,0.210005,55.040428
2022-11-13 16:10:41.174,BTCBUSD,BTC,BUSD,False,-2.689,16633.84935105,1.681323e+04,1.692682e+04,1.621379e+04,1.636108e+04,...,4.650243e+09,3.040404e+09,5.263449e+10,15005.562026,NaN,NaN,0.011124,92.225399,NaN,5.460996
2022-11-13 16:10:41.174,DASHBUSD,DASH,BUSD,False,-1.627,33.71344115,3.380000e+01,3.469000e+01,3.273000e+01,3.325000e+01,...,3.987286e+06,1.986541e+04,2.606793e+04,33.370000,929.370359,930.574450,0.179856,39.260034,0.129392,43.248344


## Pre-screen buyable base assets in pairs.

In [6]:
from cryptocurrency.conversion_table import get_new_tickers
tickers_list = get_new_tickers(conversion_table=conversion_table)
len(tickers_list)

1434

## Locally (saving time) convert price from an asset to another using existing pairs.

In [7]:
from cryptocurrency.trade import select_asset_with_biggest_wallet
from_asset, converted_quantity, quantity = \
    select_asset_with_biggest_wallet(client=client, 
                                     conversion_table=conversion_table, 
                                     exchange_info=exchange_info, 
                                     as_pair=True)
from_asset, converted_quantity, quantity

('BUSD', '10.2502', 10.238)

## Trade.

In [8]:
import pandas as pd

def get_conversion_table_2(conversion_table_file_name='crypto_logs/conversion_table.txt'):
    return pd.read_csv(conversion_table_file_name, index_col=0)

conversion_table_file_name = 'crypto_logs/conversion_table.txt'
conversion_table = get_conversion_table(client=client, exchange_info=exchange_info, 
                                        as_pair=True, dump_raw=True)
conversion_table = get_conversion_table_2(conversion_table_file_name=conversion_table_file_name)
conversion_table

,symbol,price_change,price_change_percent,weighted_average_price,close_shifted,close,last_volume,bid_price,bid_volume,ask_price,...,low,rolling_base_volume,rolling_quote_volume,open_time,close_time,first_ID,last_ID,count,base_asset,quote_asset
0,ETHBTC,-0.000926,-1.234,0.074565,0.075039,0.074117,0.5364,0.074113,4.0495,0.074114,...,0.073580,7.844385e+04,5.849137e+03,1668287477855,1668373877855,389742799,389956095,213297,ETH,BTC
1,LTCBTC,-0.000137,-3.812,0.003492,0.003595,0.003457,4.8470,0.003457,7.5030,0.003458,...,0.003422,1.147653e+05,4.007283e+02,1668287475231,1668373875231,85405172,85433903,28732,LTC,BTC
2,BNBBTC,0.000019,0.113,0.016815,0.016782,0.016801,0.1220,0.016801,6.8380,0.016802,...,0.016639,6.225496e+04,1.046844e+03,1668287477894,1668373877894,209736471,209832311,95841,BNB,BTC
3,NEOBTC,0.000020,5.089,0.000407,0.000394,0.000413,0.9500,0.000412,418.3100,0.000413,...,0.000380,2.599865e+05,1.058983e+02,1668287472580,1668373872580,44015860,44029555,13696,NEO,BTC
4,QTUMETH,-0.000001,-0.060,0.001653,0.001644,0.001653,98.3000,0.001649,98.6000,0.001654,...,0.001622,9.255800e+03,1.530440e+01,1668287475023,1668373875023,5339938,5340126,189,QTUM,ETH
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2121,OSMOUSDT,-0.063000,-5.303,1.163129,1.188000,1.125000,121.4900,1.125000,251.6500,1.130000,...,1.120000,4.167254e+05,4.847055e+05,1668287477112,1668373877112,391864,396918,5055,OSMO,USDT
2122,OSMOBUSD,-0.062000,-5.232,1.160451,1.179000,1.123000,868.2000,1.124000,48.1800,1.127000,...,1.122000,4.694670e+05,5.447933e+05,1668287477484,1668373877484,410216,414146,3931,OSMO,BUSD
2123,HFTBTC,0.000004,12.286,0.000035,0.000031,0.000035,1251.6000,0.000035,1229.7000,0.000035,...,0.000031,5.596584e+06,1.968265e+02,1668287476711,1668373876711,199321,214913,15593,HFT,BTC
2124,HFTBUSD,0.048200,9.275,0.581408,0.519700,0.567900,17994.5000,0.567900,4215.6000,0.568200,...,0.512200,4.228287e+07,2.458359e+07,1668287477995,1668373877995,1126856,1207765,80910,HFT,BUSD


In [9]:
to_asset = 'BTC'
priority = 'fees'

from cryptocurrency.trade import trade
import pandas as pd

def get_conversion_table_2(conversion_table_file_name='crypto_logs/conversion_table.txt'):
    return pd.read_csv(conversion_table_file_name, index_col=0)

conversion_table_file_name = 'crypto_logs/conversion_table.txt'
conversion_table = get_conversion_table(client=client, exchange_info=exchange_info, as_pair=True)
request = trade(client=client, to_asset=to_asset, conversion_table=conversion_table, 
                exchange_info=exchange_info, priority=priority)
from_asset = to_asset
request

[('BTC', 'BUSD')]


{'symbol': 'BTCBUSD',
 'orderId': 7051760832,
 'orderListId': -1,
 'clientOrderId': 'lCv3HMtYhzwecNnWHbrRrN',
 'transactTime': 1668373950520,
 'price': '0.00000000',
 'origQty': '0.00062000',
 'executedQty': '0.00062000',
 'cummulativeQuoteQty': '10.12458140',
 'status': 'FILLED',
 'timeInForce': 'GTC',
 'type': 'MARKET',
 'side': 'BUY',
 'fills': [{'price': '16329.97000000',
   'qty': '0.00062000',
   'commission': '0.00000000',
   'commissionAsset': 'BNB',
   'tradeId': 617437828}]}

## Main loop buying screened assets in pairs which are connected to the wallet holdings.

In [14]:
priority = 'fees'
sell_asset = 'BUSD'
keys_file = 'contabo_keys.txt'
#crypto_output_log_screened = '~/workspace/crypto_logs/crypto_output_log_15s_screened.txt'
crypto_output_log_screened_1min = '~/workspace/crypto_logs/crypto_output_log_1min_screened.txt'
crypto_output_log_screened_30min = '~/workspace/crypto_logs/crypto_output_log_30min_screened.txt'
crypto_output_log_screened_1h = '~/workspace/crypto_logs/crypto_output_log_1h_screened.txt'
crypto_output_log_screened_1d = '~/workspace/crypto_logs/crypto_output_log_1d_screened.txt'

with open(keys_file, 'r') as f:
    password = f.readline().replace('\n', '')

import sys
import time
import pandas as pd
import paramiko

if sys.version_info[0] < 3:
    from StringIO import StringIO
else:
    from io import StringIO

ssh = paramiko.SSHClient()
ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())

ssh.connect('154.12.239.24', username='sam', port=22, password=password)

def get_tradable_pairs(crypto_output_log_screened='~/workspace/crypto_logs/crypto_output_log_1d_screened.txt'):
    ssh_stdin, ssh_stdout, ssh_stderr = ssh.exec_command('cat {}'.format(crypto_output_log_screened))
    tradable_pairs = pd.read_csv(StringIO(str(ssh_stdout.read().decode("utf-8"))))
    return tradable_pairs

def choose_to_asset_to_buy(from_asset, to_asset):
    tradable_pairs = get_tradable_pairs(crypto_output_log_screened=crypto_output_log_screened_1h)
    if tradable_pairs.shape[0] < 1:
        to_asset = sell_asset
    else:
        print('.', end='')
        tradable_assets_30min = set(tradable_pairs['symbol'].tolist())
        tradable_pairs = get_tradable_pairs(crypto_output_log_screened=crypto_output_log_screened_1min)
        if tradable_pairs.shape[0] < 1:
            to_asset = sell_asset
        else:
            tradable_assets_1min = set(tradable_pairs['symbol'].tolist())
            tradable_assets = list(tradable_assets_1min & tradable_assets_30min)
            if len(tradable_assets) < 1:
                to_asset = sell_asset
            else:
                latest_asset = tradable_assets[-1]
                if from_asset not in tradable_assets:
                    to_asset = latest_asset
    return to_asset

def choose_to_asset_to_sell(from_asset, to_asset):
    tradable_pairs = get_tradable_pairs(crypto_output_log_screened=crypto_output_log_screened_1min)
    if tradable_pairs.shape[0] < 1:
        to_asset = sell_asset
    else:
        print('.', end='')
        tradable_assets = list(set(tradable_pairs['symbol'].tolist()))
        latest_asset = tradable_assets[-1]
        if from_asset not in tradable_assets:
            to_asset = latest_asset
    return to_asset

def choose_to_asset(from_asset, to_asset, crypto_output_log_screened=crypto_output_log_screened_1min):
    tradable_pairs = get_tradable_pairs(crypto_output_log_screened=crypto_output_log_screened)
    if tradable_pairs.shape[0] < 1:
        to_asset = sell_asset
    else:
        print('.', end='')
        tradable_assets = list(set(tradable_pairs['symbol'].tolist()))
        latest_asset = tradable_assets[-1]
        if from_asset not in tradable_assets:
            to_asset = latest_asset
    return to_asset
    

while True:
    #to_asset = choose_to_asset(from_asset, to_asset, 
    #                           crypto_output_log_screened=crypto_output_log_screened_1h)
    if from_asset != sell_asset:
        to_asset = choose_to_asset_to_buy(from_asset, to_asset)
    else:
        to_asset = choose_to_asset_to_sell(from_asset, to_asset)

    if from_asset != to_asset:
        conversion_table = get_conversion_table(client=client, exchange_info=exchange_info, as_pair=True)
        from_asset, converted_quantity, quantity = \
            select_asset_with_biggest_wallet(client=client, conversion_table=conversion_table, 
                                             exchange_info=exchange_info)
        request = trade(client=client, to_asset=to_asset, conversion_table=conversion_table, 
                        exchange_info=exchange_info, priority=priority)
        if request is not None:
            if to_asset != sell_asset:
                pair_bought = request['symbol']
                price_when_bought = float(request['fills'][0]['price'])
            else:
                asset_to_delete = from_asset
            from_asset = to_asset

    time.sleep(5)

ssh.close()

[('BTC', 'BUSD')]
.[('DAR', 'BUSD')]
.[('DAR', 'BUSD'), ('C98', 'BUSD')]
........................................................[('C98', 'BUSD'), ('PERP', 'BUSD')]
...............................................................................................................................

KeyboardInterrupt: 

In [ ]:
take_profit = None
stop_loss = None
sell_asset = 'BTC'
bought_asset = None
asset_to_delete = None
keys_file = 'contabo_keys.txt'
crypto_input_log_screened = 'crypto_logs/crypto_input_log_15s_screened.txt'
crypto_output_log_screened = '~/workspace/crypto_logs/crypto_output_log_1min_screened.txt'

with open(keys_file, 'r') as f:
    password = f.readline().replace('\n', '')

from cryptocurrency.authentication import Cryptocurrency_authenticator
from cryptocurrency.conversion import get_base_asset_from_pair
from cryptocurrency.exchange import Cryptocurrency_exchange

import sys
import time
import pandas as pd
import paramiko

if sys.version_info[0] < 3:
    from StringIO import StringIO
else:
    from io import StringIO

ssh = paramiko.SSHClient()
ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())

ssh.connect('154.12.239.24', username='sam', port=22, password=password)

client = Cryptocurrency_authenticator(use_keys=False, testnet=False).spot_client
exchange_info = Cryptocurrency_exchange(client=client, directory='crypto_logs').info

conversion_table = get_conversion_table(client=client, exchange_info=exchange_info, as_pair=True)
#conversion_table = get_conversion_table_2(conversion_table_file_name=conversion_table_file_name)
from_asset, converted_quantity, quantity = \
    select_asset_with_biggest_wallet(client=client, conversion_table=conversion_table, 
                                     exchange_info=exchange_info)

while True:
    try:
        if asset_to_delete is None:
            #tradable_assets = pd.read_csv(crypto_output_log_screened, index_col=0)['base_asset']
            ssh_stdin, ssh_stdout, ssh_stderr = ssh.exec_command('cat {}'.format(crypto_output_log_screened))
            tradable_pairs = pd.read_csv(StringIO(str(ssh_stdout.read().decode("utf-8"))))
            tradable_assets = tradable_pairs['symbol'].apply(lambda pair: get_base_asset_from_pair(pair, 
                                                                                                   exchange_info))
            if tradable_assets.size < 1:
                to_asset = sell_asset
            else:
                tradable_assets = list(set(tradable_assets.tolist()))
                latest_asset = tradable_assets[-1]
                if from_asset not in tradable_assets:
                    to_asset = latest_asset

            if from_asset != to_asset:
                conversion_table = get_conversion_table(client=client, exchange_info=exchange_info, as_pair=True)
                #conversion_table = get_conversion_table_2(conversion_table_file_name=conversion_table_file_name)
                from_asset, converted_quantity, quantity = \
                    select_asset_with_biggest_wallet(client=client, conversion_table=conversion_table, 
                                                     exchange_info=exchange_info)
                request = trade(client=client, to_asset=to_asset, conversion_table=conversion_table, 
                                exchange_info=exchange_info, priority=priority)
                if request is not None:
                    if to_asset != sell_asset:
                        pair_bought = request['symbol']
                        price_when_bought = float(request['fills'][0]['price'])
                    else:
                        asset_to_delete = from_asset
                    from_asset = to_asset
            elif bought_asset is not None:
                conversion_table = get_conversion_table(client=client, exchange_info=exchange_info, as_pair=True)
                #conversion_table = get_conversion_table_2(conversion_table_file_name=conversion_table_file_name)
                price_now = conversion_table[conversion_table['symbol'] == bought_asset]['lastPrice'].iat[0]
                percent_gain = ((price_now - price_when_bought) / price_when_bought) * 100.0
                if stop_loss is not None:
                    if percent_gain <= -stop_loss:
                        to_asset = latest_asset
                elif take_profit is not None:
                    if percent_gain >= take_profit:
                        to_asset = latest_asset
                else:
                    asset_to_delete = None
            time.sleep(2)
        else:
            input_pairs = pd.read_csv(crypto_input_log_screened, index_col=0)
            input_pairs = input_pairs[input_pairs['base_asset'] != asset_to_delete]
            input_pairs.to_csv(crypto_input_log_screened)
            asset_to_delete = None
    except (IndexError, pd.errors.EmptyDataError):
        time.sleep(0.5)
        continue

ssh.close()

In [ ]:
take_profit = None
stop_loss = None
sell_asset = 'BTC'
bought_asset = None
asset_to_delete = None
keys_file = 'contabo_keys.txt'
crypto_input_log_screened = 'crypto_logs/crypto_input_log_15s_screened.txt'
crypto_output_log_screened = '~/workspace/crypto_logs/crypto_output_log_1min_screened.txt'

with open(keys_file, 'r') as f:
    password = f.readline().replace('\n', '')

from cryptocurrency.authentication import Cryptocurrency_authenticator
from cryptocurrency.conversion import get_base_asset_from_pair
from cryptocurrency.exchange import Cryptocurrency_exchange

import sys
import time
import pandas as pd
import paramiko

if sys.version_info[0] < 3:
    from StringIO import StringIO
else:
    from io import StringIO

ssh = paramiko.SSHClient()
ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())

ssh.connect('154.12.239.24', username='sam', port=22, password=password)

client = Cryptocurrency_authenticator(use_keys=False, testnet=False).spot_client
exchange_info = Cryptocurrency_exchange(client=client, directory='crypto_logs').info

conversion_table = get_conversion_table(client=client, exchange_info=exchange_info, as_pair=True)
#conversion_table = get_conversion_table_2(conversion_table_file_name=conversion_table_file_name)
from_asset, converted_quantity, quantity = \
    select_asset_with_biggest_wallet(client=client, conversion_table=conversion_table, 
                                     exchange_info=exchange_info)

while True:
    try:
        if asset_to_delete is None:
            #tradable_assets = pd.read_csv(crypto_output_log_screened, index_col=0)['base_asset']
            ssh_stdin, ssh_stdout, ssh_stderr = ssh.exec_command('cat {}'.format(crypto_output_log_screened))
            tradable_pairs = pd.read_csv(StringIO(str(ssh_stdout.read().decode("utf-8"))))
            tradable_assets = tradable_pairs['symbol'].apply(lambda pair: get_base_asset_from_pair(pair, 
                                                                                                   exchange_info))
            if tradable_assets.size < 1:
                to_asset = sell_asset
            else:
                tradable_assets = list(set(tradable_assets.tolist()))
                latest_asset = tradable_assets[-1]
                if from_asset not in tradable_assets:
                    to_asset = latest_asset

            if from_asset != to_asset:
                conversion_table = get_conversion_table(client=client, exchange_info=exchange_info, as_pair=True)
                #conversion_table = get_conversion_table_2(conversion_table_file_name=conversion_table_file_name)
                from_asset, converted_quantity, quantity = \
                    select_asset_with_biggest_wallet(client=client, conversion_table=conversion_table, 
                                                     exchange_info=exchange_info)
                request = trade(client=client, to_asset=to_asset, conversion_table=conversion_table, 
                                exchange_info=exchange_info, priority=priority)
                if request is not None:
                    if to_asset != sell_asset:
                        pair_bought = request['symbol']
                        price_when_bought = float(request['fills'][0]['price'])
                    else:
                        asset_to_delete = from_asset
                    from_asset = to_asset
            elif bought_asset is not None:
                conversion_table = get_conversion_table(client=client, exchange_info=exchange_info, as_pair=True)
                #conversion_table = get_conversion_table_2(conversion_table_file_name=conversion_table_file_name)
                price_now = conversion_table[conversion_table['symbol'] == bought_asset]['lastPrice'].iloc[0]
                percent_gain = ((price_now - price_when_bought) / price_when_bought) * 100.0
                if stop_loss is not None:
                    if percent_gain <= -stop_loss:
                        to_asset = latest_asset
                elif take_profit is not None:
                    if percent_gain >= take_profit:
                        to_asset = latest_asset
                else:
                    asset_to_delete = None
            time.sleep(2)
        else:
            input_pairs = pd.read_csv(crypto_input_log_screened, index_col=0)
            input_pairs = input_pairs[input_pairs['base_asset'] != asset_to_delete]
            input_pairs.to_csv(crypto_input_log_screened)
            asset_to_delete = None
    except (IndexError, pd.errors.EmptyDataError):
        time.sleep(0.5)
        continue

ssh.close()

In [ ]:
'''
take_profit = 10.0
stop_loss = None
sell_asset = 'BTC'
bought_asset = None
asset_to_delete = None
crypto_input_log_screened = 'crypto_logs/crypto_input_log_1min_screened.txt'
crypto_output_log_screened = 'crypto_logs/crypto_output_log_30min_screened.txt'

from time import sleep
from cryptocurrency.authentication import Cryptocurrency_authenticator
from cryptocurrency.exchange import Cryptocurrency_exchange

client = Cryptocurrency_authenticator(use_keys=False, testnet=False).spot_client
exchange_info = Cryptocurrency_exchange(client=client, directory='crypto_logs').info

#conversion_table = get_conversion_table(client=client, exchange_info=exchange_info, as_pair=True)
conversion_table = get_conversion_table_2(conversion_table_file_name=conversion_table_file_name)
from_asset, converted_quantity, quantity = \
    select_asset_with_biggest_wallet(client=client, conversion_table=conversion_table, 
                                     exchange_info=exchange_info)

while True:
    try:
        if asset_to_delete is None:
            tradable_assets = pd.read_csv(crypto_output_log_screened, index_col=0)['base_asset']
            if tradable_assets.size < 1:
                to_asset = sell_asset
            else:
                tradable_assets = list(set(tradable_assets.tolist()))
                latest_asset = tradable_assets[-1]
                if from_asset not in tradable_assets:
                    to_asset = latest_asset

            if from_asset != to_asset:
                #conversion_table = get_conversion_table(client=client, exchange_info=exchange_info, as_pair=True)
                conversion_table = get_conversion_table_2(conversion_table_file_name=conversion_table_file_name)
                request = trade(client=client, to_asset=to_asset, conversion_table=conversion_table, 
                                exchange_info=exchange_info, priority=priority)
                if request is not None:
                    if to_asset != sell_asset:
                        pair_bought = request['symbol']
                        price_when_bought = float(request['fills'][0]['price'])
                    else:
                        asset_to_delete = from_asset
                    from_asset = to_asset
            elif bought_asset is not None:
                #conversion_table = get_conversion_table(client=client, exchange_info=exchange_info, as_pair=True)
                conversion_table = get_conversion_table_2(conversion_table_file_name=conversion_table_file_name)
                price_now = conversion_table[conversion_table['symbol'] == bought_asset]['lastPrice'].iloc[0]
                percent_gain = ((price_now - price_when_bought) / price_when_bought) * 100.0
                if stop_loss is not None:
                    if percent_gain <= -stop_loss:
                        to_asset = latest_asset
                elif take_profit is not None:
                    if percent_gain >= take_profit:
                        to_asset = latest_asset
                else:
                    asset_to_delete = None
            sleep(5)
        else:
            input_pairs = pd.read_csv(crypto_input_log_screened, index_col=0)
            input_pairs = input_pairs[input_pairs['base_asset'] != asset_to_delete]
            input_pairs.to_csv(crypto_input_log_screened)
            asset_to_delete = None
    except (IndexError, pd.errors.EmptyDataError):
        sleep(0.5)
        continue
'''

In [ ]:
keys_file = 'contabo_keys.txt'
output_log_screened = '~/workspace/crypto_logs/crypto_output_log_1min_screened.txt'

with open(keys_file, 'r') as f:
    password = f.readline().replace('\n', '')

import sys
import time
import pandas as pd
import paramiko

if sys.version_info[0] < 3:
    from StringIO import StringIO
else:
    from io import StringIO

ssh = paramiko.SSHClient()
ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())

ssh.connect('154.12.239.24', username='sam', port=22, password=password)

while True:
    ssh_stdin, ssh_stdout, ssh_stderr = ssh.exec_command('cat {}'.format(crypto_output_log_screened))

    df = pd.read_csv(StringIO(str(ssh_stdout.read().decode("utf-8"))))

    if df.shape[0] < 1:
        print('None')
    else:
        print(df)

    time.sleep(5)

ssh.close()

In [ ]:
try:
    tradable_pairs = pd.read_csv(crypto_output_log_screened, index_col=0)['symbol']
    conversion_table = get_conversion_table_2(conversion_table_file_name=conversion_table_file_name)
    tradable_assets = [get_base_asset_from_pair(tradable_pair, exchange_info=exchange_info) 
                       for tradable_pair in tradable_pairs.tolist()]
    tradable_assets = list(set(tradable_assets))
except pd.errors.EmptyDataError:
    sleep(0.5)
    tradable_assets = []
tradable_assets

In [ ]:
def get_highest_daily_volume_pair_associated_with_base_asset(base_asset, conversion_table, exchange_info):
    pairs = exchange_info[exchange_info['baseAsset'] == base_asset]['symbol'].tolist()
    filtered_pairs = conversion_table[conversion_table['symbol'].isin(pairs)]
    filtered_pairs = filtered_pairs.sort_values(by=['volume'], ascending=False)
    return filtered_pairs['symbol'].iloc[0]

pair = get_highest_daily_volume_pair_associated_with_base_asset(base_asset='LSK', 
                                                                conversion_table=conversion_table, 
                                                                exchange_info=exchange_info)

pair

In [ ]:
def get_highest_daily_volume_pair_associated_with_base_asset(base_asset, conversion_table, exchange_info):
    pairs = exchange_info[exchange_info['baseAsset'] == base_asset]['symbol'].tolist()
    filtered_pairs = conversion_table[conversion_table['symbol'].isin(pairs)]
    filtered_pairs = filtered_pairs.sort_values(by=['volume'], ascending=False)
    return filtered_pairs['symbol'].iloc[0]

crypto_input_log_screened_name = 'crypto_logs/crypto_input_log_1min_screened.txt'
dataset = pd.read_csv(crypto_input_log_screened_name, index_col=0)

dataset['base_asset'] = dataset['symbol'].apply(lambda x: get_base_asset_from_pair(x, exchange_info=exchange_info))
dataset['symbol'] = dataset['base_asset'].apply(lambda x: get_highest_daily_volume_pair_associated_with_base_asset(
                                                              base_asset=x, 
                                                              conversion_table=conversion_table, 
                                                              exchange_info=exchange_info
                                                          ))
dataset = dataset.drop_duplicates(subset=['symbol'], keep='last')
dataset